In [10]:
%pip install beautifulsoup4 requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import requests
from bs4 import BeautifulSoup
import time
import random
import json
import os

In [12]:
BASE_URL = "http://www.lily-chou-chou.jp/holic/bbs/htm/index2_mddl.php?ofst="
START_PAGES = 426828 #start from 427032th pages back to 1
LIMIT = 425900 #upto 427000th pages

In [13]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

In [14]:
# ===== NETWORK =====
CONNECT_TIMEOUT = 5
READ_TIMEOUT = 120     # offset sâu cần read timeout dài

# ===== SPEED CONTROL =====
BASE_SLEEP = 3.0     # tối thiểu 10s / request
JITTER = 5.0          # random thêm 0–5s
MAX_RETRY = 2         # KHÔNG retry nhiều

# ===== BLOCK CONTROL =====
FAIL_THRESHOLD = 3
BLOCK_PAUSE = 5 * 60   # 5 phút

# ===== WINDOW =====
WINDOW_SIZE = 100        # crawl rất sâu → window nhỏ

# ===== ENCODING =====
PAGE_ENCODING = "euc-jp"


In [15]:
PROGRESS_FILE = "crawl_progress.json" # File lưu trạng thái offset và data

In [16]:
def load_checkpoint():
    if os.path.exists(PROGRESS_FILE):
        try:
            with open(PROGRESS_FILE, 'r', encoding='utf-8') as f:
                checkpoint = json.load(f)
                print(f"🔄 Tìm thấy tiến trình cũ: Offset cuối cùng là {checkpoint['last_offset']}")
                return checkpoint['last_offset'], checkpoint['post_list']
        except Exception as e:
            print(f"⚠️ Lỗi đọc file checkpoint, bắt đầu mới: {e}")
    return START_PAGES, []

def save_checkpoint(last_offset, post_list):
    checkpoint = {
        "last_offset": last_offset,
        "post_list": post_list
    }
    with open(PROGRESS_FILE, 'w', encoding='utf-8') as f:
        json.dump(checkpoint, f, ensure_ascii=False, indent=4)
    print(f"💾 Đã lưu checkpoint tại offset {last_offset}. Tổng số bài: {len(post_list)}")

In [17]:
POST_LIST = []

In [18]:

session = requests.Session()
session.headers.update(headers)

In [19]:
def crawl_offset(ofst: int) -> bool:
    url = BASE_URL + str(ofst)
    print(f"[CRAWLING] ofst={ofst}")

    response = None

    for attempt in range(MAX_RETRY + 1):
        try:
            response = session.get(
                url,
                timeout=(CONNECT_TIMEOUT, READ_TIMEOUT)
            )
            break
        except requests.exceptions.ReadTimeout:
            wait = BASE_SLEEP * (attempt + 1)
            print(f"[TIMEOUT] ofst={ofst}, wait {wait:.1f}s")
            time.sleep(wait)
        except requests.exceptions.RequestException as e:
            print(f"[ERROR] ofst={ofst}: {e}")
            return False

    if response is None or response.status_code != 200:
        print(f"[SKIP] ofst={ofst}")
        return False

    if len(response.content) < 800:
        print(f"[SKIP] tiny response at ofst={ofst}")
        return False

    response.encoding = PAGE_ENCODING
    soup = BeautifulSoup(response.text, 'html.parser')
    all_trs = soup.find_all('tr')

    current_post = {}

    for tr in all_trs:
        # ===== HEADER =====
        header_td = tr.find('td', attrs={'width': '450', 'colspan': '2'})
        if header_td and not header_td.has_attr('align'):
            fonts = header_td.find_all('font')
            if len(fonts) >= 4:
                current_post = {}
                current_post['user'] = fonts[1].get_text(strip=True)
                current_post['post_name'] = fonts[3].get_text(strip=True)

        # ===== CONTENT =====
        content_td = tr.find(
            'td',
            attrs={'width': '450', 'colspan': '2', 'align': 'left'}
        )
        if content_td and 'user' in current_post:
            fonts = content_td.find_all('font')
            if fonts:
                current_post['post_content'] = fonts[0].get_text(strip=True)
                POST_LIST.append(current_post)
                current_post = {}

    return True


In [ ]:
# Tải tiến trình cũ nếu có
current_start_offset, POST_LIST = load_checkpoint()

consecutive_failures = 0

# Duyệt theo từng window
for window_start in range(current_start_offset, LIMIT, -WINDOW_SIZE):
    window_end = max(window_start - WINDOW_SIZE, LIMIT)
    
    print(f"\n[WINDOW] {window_start} → {window_end}")

    for ofst in range(window_start, window_end, -1):
        ok = crawl_offset(ofst)

        if not ok:
            consecutive_failures += 1
            print(f"[FAIL] consecutive={consecutive_failures}")
        else:
            consecutive_failures = 0

        # Xử lý khi bị block
        if consecutive_failures >= FAIL_THRESHOLD:
            print("\n🚨 [BLOCK DETECTED]")
            # Lưu lại vị trí hiện tại trước khi nghỉ
            save_checkpoint(ofst, POST_LIST) 
            print(f"⏸ Tạm nghỉ {BLOCK_PAUSE//60} phút...")
            time.sleep(BLOCK_PAUSE)
            consecutive_failures = 0
            break

        sleep_time = BASE_SLEEP + random.random() * JITTER
        time.sleep(sleep_time)

    # SAU MỖI WINDOW: Ghi đè vào file JSON để đảm bảo không mất dữ liệu
    print(f"\n[WINDOW DONE] Đã xong window tới offset {window_end}")
    save_checkpoint(window_end, POST_LIST)
    
    # Nghỉ dài giữa các window
    print(f"☕ Nghỉ dài giữa các window... {BLOCK_PAUSE//60} phút")
    time.sleep(BLOCK_PAUSE)

print(f"[DONE] Hoàn tất crawl. Tổng số bài: {len(POST_LIST)}")


[WINDOW] 426828 → 426728
[CRAWLING] ofst=426828
[CRAWLING] ofst=426827
[CRAWLING] ofst=426826
[CRAWLING] ofst=426825
[CRAWLING] ofst=426824
[CRAWLING] ofst=426823
[CRAWLING] ofst=426822
[CRAWLING] ofst=426821
[CRAWLING] ofst=426820
[CRAWLING] ofst=426819
[CRAWLING] ofst=426818
[CRAWLING] ofst=426817
[CRAWLING] ofst=426816
[CRAWLING] ofst=426815
[CRAWLING] ofst=426814
[CRAWLING] ofst=426813
[CRAWLING] ofst=426812
[CRAWLING] ofst=426811
[CRAWLING] ofst=426810
[CRAWLING] ofst=426809
[CRAWLING] ofst=426808
[CRAWLING] ofst=426807
[CRAWLING] ofst=426806
[CRAWLING] ofst=426805
[CRAWLING] ofst=426804
[CRAWLING] ofst=426803
[CRAWLING] ofst=426802
[CRAWLING] ofst=426801
[CRAWLING] ofst=426800
[CRAWLING] ofst=426799
[CRAWLING] ofst=426798
[CRAWLING] ofst=426797
[CRAWLING] ofst=426796
[CRAWLING] ofst=426795
[CRAWLING] ofst=426794
[CRAWLING] ofst=426793
[CRAWLING] ofst=426792
[CRAWLING] ofst=426791
[CRAWLING] ofst=426790
[CRAWLING] ofst=426789
[CRAWLING] ofst=426788
[CRAWLING] ofst=426787
[CRAWLIN

In [ ]:
import json
with open('lily_chou_chou_posts.json', 'w', encoding='utf-8') as f:
    json.dump(POST_LIST, f, ensure_ascii=False, indent=4)